In [1]:
from rouge import Rouge, FilesRouge
import pandas as pd
import numpy as np

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)

In [2]:
print(scores)

[{'rouge-1': {'f': 0.49411764217577864, 'p': 0.5833333333333334, 'r': 0.42857142857142855}, 'rouge-2': {'f': 0.23423422957552154, 'p': 0.3170731707317073, 'r': 0.18571428571428572}, 'rouge-l': {'f': 0.42751590030718895, 'p': 0.5277777777777778, 'r': 0.3877551020408163}}]


In [3]:
d = {'pred': [hypothesis], 'actuals': [reference]}

In [76]:
# read in actual notes
notes = pd.read_csv("Z:/final_data/scored_data/notes_scored.csv",index_col=0,keep_default_na=False)
notes.columns = ['scored_summaries']

In [77]:
notes.head()

,scored_summaries
18,Increase in diffuse pulmonary opacities which ...
19,Increase in diffuse pulmonary opacities which ...
20,Diffuse nonspecific interstitial disease of un...
21,Diffuse nonspecific interstitial disease of un...
22,Right chest tube in place with no significant ...


In [78]:
# join notes to the test set
test_dta = pd.read_csv("Z:/final_data/cohorts_merged_test.csv",keep_default_na=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,325) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [79]:
# join in the columns 
# test_dta.join(notes)
scored_df = pd.merge(test_dta, notes, left_index=True, right_index=True,how="left")

In [80]:
scored_df.loc[23:25]

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,visit_status_2,...,comparison,findings,impressions,elecsig,patient_raw,rawtext,encounter_id_diagnosed,date_diagnosed,ild_status,scored_summaries
23,114,66,F,White,Not Hispanic or Latino,,9495002,I,No Show,Completed,...,Multiple recent radiographs.,Chest tube at the right apex with no significa...,Right chest tube in place with no significant ...,**DATE<[**2019-07-22**]> 8:55 AM **NAME<JJJ K...,9495002_317253,"XR PORT CHEST 1V, **DATE<[**2019-07-22**]> 4:4...",,,0,Right chest tube in place with no significant ...
24,114,66,F,White,Not Hispanic or Latino,,9495002,I,No Show,Completed,...,,,,,,,6221749,2014-12-06,1,NaN
25,114,66,F,White,Not Hispanic or Latino,,9495002,I,No Show,Completed,...,,,,,,,,,0,NaN


# now let's get a full function to take any given df, define rows of pred and actuals and score them, and append to a df

In [128]:
def score_data(df, predicted, actuals):
    printcounter = 0
    for index,row in df.iterrows():
#         print(printcounter)
        printcounter += 1
        if(printcounter % 1000)==0:
            print("[info] 1000 iterations")
#         print("[info] index #: ",index)
#         print(row[predicted])
#         print(row[actuals])
        if str(row[predicted])!="nan":
            print("[info] this was not a null row. Logging index")
            indexlist.append(index)
            rougescore = rouge.get_scores(row[predicted],row[actuals])
            tmpdf = pd.DataFrame(rougescore)
            print("[info] now appending to list rouge values")
            for row in tmpdf:
                scorelist.append(tmpdf[row].apply(pd.Series).values[0])
        else:
            print("[info] this was a null row. skipped.")

# try on actuals

In [ ]:
# get np array now of rouge scores
scorelist = []
indexlist = []
score_data(scored_df,'scored_summaries','findings')
print(scorelist)
print(indexlist)

[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was not a null row. Logging index
[info] now appending to 

[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a n

[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. 

[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipp

[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this wa

...

[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values


[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[

[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[

[info] now appending to list rouge values
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a n

[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was not a null row. Logging index
[info] now appending to list rouge values
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a null row. skipped.
[info] this was a n

In [120]:
# assign
rouged_df = pd.DataFrame(np.array(scorelist).reshape(int(len(scorelist)/3),9),columns=['rouge-1-f','rouge-1-p','rouge-1-r','rouge-2-f','rouge-2-p','rouge-2-r','rouge-l-f','rouge-2-p','rouge-2-r'],index=indexlist)
rouged_df.head(n=10)


,rouge-1-f,rouge-1-p,rouge-1-r,rouge-2-f,rouge-2-p,rouge-2-r,rouge-l-f,rouge-2-p,rouge-2-r
18,0.384615,0.333333,0.454545,0.296296,0.235294,0.400000,0.367616,0.333333,0.454545
19,0.384615,0.333333,0.454545,0.296296,0.235294,0.400000,0.367616,0.333333,0.454545
20,0.228571,0.363636,0.166667,0.121212,0.200000,0.086957,0.183966,0.363636,0.166667
21,0.228571,0.363636,0.166667,0.121212,0.200000,0.086957,0.183966,0.363636,0.166667
22,0.322581,0.384615,0.277778,0.206897,0.250000,0.176471,0.307012,0.384615,0.277778
23,0.322581,0.384615,0.277778,0.206897,0.250000,0.176471,0.307012,0.384615,0.277778
